In [3]:
import pandas as pd
pd.set_option("display.max.columns", None)
anime_master_df = pd.read_csv(r"C:\Users\PC\Desktop\top_anime_dataset_v2.csv")
anime_master_df.head()

,anime_id,anime_url,image_url,name,english_name,japanese_names,score,genres,themes,demographics,synopsis,type,episodes,premiered,producers,studios,source,duration,rating,rank,popularity,favorites,scored_by,members
0,52991,https://myanimelist.net/anime/52991/Sousou_no_...,https://cdn.myanimelist.net/images/anime/1015/...,Sousou no Frieren,Frieren: Beyond Journey's End,葬送のフリーレン,9.31,"Adventure, Drama, Fantasy",NaN,Shounen,During their decade-long quest to defeat the D...,TV,28.0,fall 2023,"Aniplex, Dentsu, Shogakukan-Shueisha Productio...",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,1.0,160,63387,605478,1038347
1,5114,https://myanimelist.net/anime/5114/Fullmetal_A...,https://cdn.myanimelist.net/images/anime/1208/...,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,9.10,"Action, Adventure, Drama, Fantasy",Military,Shounen,After a horrific alchemy experiment goes wrong...,TV,64.0,spring 2009,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,24 min per ep,R - 17+ (violence & profanity),2.0,3,232007,2196965,3484360
2,9253,https://myanimelist.net/anime/9253/Steins_Gate,https://cdn.myanimelist.net/images/anime/1935/...,Steins;Gate,Steins;Gate,STEINS;GATE,9.07,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",NaN,Eccentric scientist Rintarou Okabe has a never...,TV,24.0,spring 2011,"Frontier Works, Media Factory, Kadokawa Shoten...",White Fox,Visual novel,24 min per ep,PG-13 - Teens 13 or older,3.0,14,194580,1450481,2669303
3,60022,https://myanimelist.net/anime/60022/One_Piece_...,https://cdn.myanimelist.net/images/anime/1455/...,One Piece Fan Letter,NaN,ONE PIECE FAN LETTER,9.06,"Action, Adventure, Fantasy",NaN,Shounen,Although the golden age of piracy is about to ...,TV Special,1.0,NaN,NaN,Toei Animation,Light novel,24 min,PG-13 - Teens 13 or older,4.0,2334,1976,69555,92128
4,38524,https://myanimelist.net/anime/38524/Shingeki_n...,https://cdn.myanimelist.net/images/anime/1517/...,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,9.05,"Action, Drama, Suspense","Gore, Military, Survival",Shounen,Seeking to restore humanity's diminishing hope...,TV,10.0,spring 2019,"Production I.G, Dentsu, Mainichi Broadcasting ...",Wit Studio,Manga,23 min per ep,R - 17+ (violence & profanity),5.0,21,60535,1671960,2409440


In [4]:
print(anime_master_df['themes'].isnull().sum())
anime_master_df.info()

5077
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   anime_id        15000 non-null  int64  
 1   anime_url       15000 non-null  object 
 2   image_url       15000 non-null  object 
 3   name            15000 non-null  object 
 4   english_name    8358 non-null   object 
 5   japanese_names  14954 non-null  object 
 6   score           15000 non-null  float64
 7   genres          13399 non-null  object 
 8   themes          9923 non-null   object 
 9   demographics    4408 non-null   object 
 10  synopsis        14532 non-null  object 
 11  type            14999 non-null  object 
 12  episodes        14888 non-null  float64
 13  premiered       4684 non-null   object 
 14  producers       9571 non-null   object 
 15  studios         12621 non-null  object 
 16  source          15000 non-null  object 
 17  duration        15000 non-

In [5]:
anime_master_df.head()

,anime_id,anime_url,image_url,name,english_name,japanese_names,score,genres,themes,demographics,synopsis,type,episodes,premiered,producers,studios,source,duration,rating,rank,popularity,favorites,scored_by,members
0,52991,https://myanimelist.net/anime/52991/Sousou_no_...,https://cdn.myanimelist.net/images/anime/1015/...,Sousou no Frieren,Frieren: Beyond Journey's End,葬送のフリーレン,9.31,"Adventure, Drama, Fantasy",NaN,Shounen,During their decade-long quest to defeat the D...,TV,28.0,fall 2023,"Aniplex, Dentsu, Shogakukan-Shueisha Productio...",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,1.0,160,63387,605478,1038347
1,5114,https://myanimelist.net/anime/5114/Fullmetal_A...,https://cdn.myanimelist.net/images/anime/1208/...,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,9.10,"Action, Adventure, Drama, Fantasy",Military,Shounen,After a horrific alchemy experiment goes wrong...,TV,64.0,spring 2009,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,24 min per ep,R - 17+ (violence & profanity),2.0,3,232007,2196965,3484360
2,9253,https://myanimelist.net/anime/9253/Steins_Gate,https://cdn.myanimelist.net/images/anime/1935/...,Steins;Gate,Steins;Gate,STEINS;GATE,9.07,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",NaN,Eccentric scientist Rintarou Okabe has a never...,TV,24.0,spring 2011,"Frontier Works, Media Factory, Kadokawa Shoten...",White Fox,Visual novel,24 min per ep,PG-13 - Teens 13 or older,3.0,14,194580,1450481,2669303
3,60022,https://myanimelist.net/anime/60022/One_Piece_...,https://cdn.myanimelist.net/images/anime/1455/...,One Piece Fan Letter,NaN,ONE PIECE FAN LETTER,9.06,"Action, Adventure, Fantasy",NaN,Shounen,Although the golden age of piracy is about to ...,TV Special,1.0,NaN,NaN,Toei Animation,Light novel,24 min,PG-13 - Teens 13 or older,4.0,2334,1976,69555,92128
4,38524,https://myanimelist.net/anime/38524/Shingeki_n...,https://cdn.myanimelist.net/images/anime/1517/...,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,9.05,"Action, Drama, Suspense","Gore, Military, Survival",Shounen,Seeking to restore humanity's diminishing hope...,TV,10.0,spring 2019,"Production I.G, Dentsu, Mainichi Broadcasting ...",Wit Studio,Manga,23 min per ep,R - 17+ (violence & profanity),5.0,21,60535,1671960,2409440


In [6]:
anime_master_df.columns

Index(['anime_id', 'anime_url', 'image_url', 'name', 'english_name',
       'japanese_names', 'score', 'genres', 'themes', 'demographics',
       'synopsis', 'type', 'episodes', 'premiered', 'producers', 'studios',
       'source', 'duration', 'rating', 'rank', 'popularity', 'favorites',
       'scored_by', 'members'],
      dtype='object')

In [7]:
anime_master_df['synopsis'][1]

'After a horrific alchemy experiment goes wrong in the Elric household, brothers Edward and Alphonse are left in a catastrophic new reality. Ignoring the alchemical principle banning human transmutation, the boys attempted to bring their recently deceased mother back to life. Instead, they suffered brutal personal loss: Alphonse\'s body disintegrated while Edward lost a leg and then sacrificed an arm to keep Alphonse\'s soul in the physical realm by binding it to a hulking suit of armor.\n\nThe brothers are rescued by their neighbor Pinako Rockbell and her granddaughter Winry. Known as a bio-mechanical engineering prodigy, Winry creates prosthetic limbs for Edward by utilizing "automail," a tough, versatile metal used in robots and combat armor. After years of training, the Elric brothers set off on a quest to restore their bodies by locating the Philosopher\'s Stone—a powerful gem that allows an alchemist to defy the traditional laws of Equivalent Exchange.\n\nAs Edward becomes an inf

In [8]:
anime_master_df['episodes'] = pd.to_numeric(anime_master_df['episodes'], errors = 'coerce')

In [9]:
selected_features = ['genres', 'themes', 'demographics', 'synopsis', 'type', 'producers', 'source']

for col in selected_features:
    anime_master_df[col] = anime_master_df[col].fillna('')

In [10]:
anime_master_df.head()

,anime_id,anime_url,image_url,name,english_name,japanese_names,score,genres,themes,demographics,synopsis,type,episodes,premiered,producers,studios,source,duration,rating,rank,popularity,favorites,scored_by,members
0,52991,https://myanimelist.net/anime/52991/Sousou_no_...,https://cdn.myanimelist.net/images/anime/1015/...,Sousou no Frieren,Frieren: Beyond Journey's End,葬送のフリーレン,9.31,"Adventure, Drama, Fantasy",,Shounen,During their decade-long quest to defeat the D...,TV,28.0,fall 2023,"Aniplex, Dentsu, Shogakukan-Shueisha Productio...",Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,1.0,160,63387,605478,1038347
1,5114,https://myanimelist.net/anime/5114/Fullmetal_A...,https://cdn.myanimelist.net/images/anime/1208/...,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,9.10,"Action, Adventure, Drama, Fantasy",Military,Shounen,After a horrific alchemy experiment goes wrong...,TV,64.0,spring 2009,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",Bones,Manga,24 min per ep,R - 17+ (violence & profanity),2.0,3,232007,2196965,3484360
2,9253,https://myanimelist.net/anime/9253/Steins_Gate,https://cdn.myanimelist.net/images/anime/1935/...,Steins;Gate,Steins;Gate,STEINS;GATE,9.07,"Drama, Sci-Fi, Suspense","Psychological, Time Travel",,Eccentric scientist Rintarou Okabe has a never...,TV,24.0,spring 2011,"Frontier Works, Media Factory, Kadokawa Shoten...",White Fox,Visual novel,24 min per ep,PG-13 - Teens 13 or older,3.0,14,194580,1450481,2669303
3,60022,https://myanimelist.net/anime/60022/One_Piece_...,https://cdn.myanimelist.net/images/anime/1455/...,One Piece Fan Letter,NaN,ONE PIECE FAN LETTER,9.06,"Action, Adventure, Fantasy",,Shounen,Although the golden age of piracy is about to ...,TV Special,1.0,NaN,,Toei Animation,Light novel,24 min,PG-13 - Teens 13 or older,4.0,2334,1976,69555,92128
4,38524,https://myanimelist.net/anime/38524/Shingeki_n...,https://cdn.myanimelist.net/images/anime/1517/...,Shingeki no Kyojin Season 3 Part 2,Attack on Titan Season 3 Part 2,進撃の巨人 Season3 Part.2,9.05,"Action, Drama, Suspense","Gore, Military, Survival",Shounen,Seeking to restore humanity's diminishing hope...,TV,10.0,spring 2019,"Production I.G, Dentsu, Mainichi Broadcasting ...",Wit Studio,Manga,23 min per ep,R - 17+ (violence & profanity),5.0,21,60535,1671960,2409440


In [11]:
anime_master_df['combined_features'] = (
    anime_master_df['genres'] + ' ' +
    anime_master_df['themes'] + ' ' +
    anime_master_df['demographics'] + ' ' +
    anime_master_df['synopsis'] + ' ' +
    anime_master_df['type'] + ' ' +
    anime_master_df['producers'] + ' ' +
    anime_master_df['source']
)

In [16]:
anime_master_df.combined_features[0]

'adventure, drama, fantasi shounen dure their decade-long quest to defeat the demon king, the member of the hero\' party—himmel himself, the priest heiter, the dwarf warrior eisen, and the elven mage frieren—forg bond through adventur and battles, creat unforgett preciou memori for most of them. however, the time that frieren spend with her comrad is equival to mere a fraction of her life, which ha last over a thousand years. when the parti disband after their victory, frieren casual return to her "usual" routin of collect spell across the continent. due to her differ sens of time, she seemingli hold no strong feel toward the experi she went through. as the year pass, frieren gradual realiz how her day in the hero\' parti truli impact her. wit the death of two of her former companions, frieren begin to regret have taken their presenc for granted; she vow to better understand human and creat real person connections. although the stori of that onc memor journey ha long ended, a new tale 

In [15]:
import re
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def preprocess(text):
    if not isinstance(text, str):
        return ""
    ps = PorterStemmer()
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n', '', text)
    text = " ".join(ps.stem(word) for word in text.split())
    
    return text

anime_master_df['combined_features'] = anime_master_df['combined_features'].apply(preprocess)

In [ ]:
anime_master_df.head()

In [17]:
tf_vec = TfidfVectorizer(stop_words = 'english')
vectorized_features = tf_vec.fit_transform(anime_master_df['combined_features'])

In [25]:
similarity = cosine_similarity(vectorized_features)

In [27]:
type(similarity)

numpy.ndarray

In [19]:
movie = input('Movie: ')

Movie:  Naruto


In [20]:
input_idx = anime_master_df[anime_master_df['name'] == movie].index[0].item()
input_idx

672

In [21]:
def recommend(movie):
    recommended_anime = []
    recommended_anime_poster = []
    input_idx = anime_master_df[anime_master_df['name'] == movie].index[0].item()
    similarity_score = list(enumerate(similarity[input_idx]))
    sorted_similarity_score = sorted(similarity_score, key = lambda x: x[1], reverse = True)

    candidates = []
    for each in sorted_similarity_score[1:11]:
        idx = each[0]
        candidates.append({
            'name': anime_master_df.iloc[idx]['name'],
            'poster': anime_master_df.iloc[idx]['image_url'],
            'score': anime_master_df.iloc[idx]['score']
        })

    candidates = sorted(candidates, key = lambda x: x['score'], reverse = True)

    recommended_anime = [m['name'] for m in candidates[:5]]
    recommended_anime_poster = [m['poster'] for m in candidates[:5]]
    
    return recommended_anime, recommended_anime_poster

In [22]:
recommend(movie)

(['Naruto: Shippuuden',
  'Naruto: Shippuuden Movie 6 - Road to Ninja',
  'Naruto: Shippuuden Movie 5 - Blood Prison',
  'Naruto: Shippuuden Movie 4 - The Lost Tower',
  'Boruto: Naruto the Movie'],
 ['https://cdn.myanimelist.net/images/anime/1565/111305.jpg',
  'https://cdn.myanimelist.net/images/anime/1620/94336.jpg',
  'https://cdn.myanimelist.net/images/anime/1500/134496.jpg',
  'https://cdn.myanimelist.net/images/anime/1479/116734.jpg',
  'https://cdn.myanimelist.net/images/anime/4/78280.jpg'])

In [ ]:
pred_list = [20, 0.975]

anime_master_df.iloc[pred_list[0]]['name']

In [ ]:
pred_list[0]

In [ ]:
similarity_matrix_score = list(enumerate(similarity_matrix[input_idx]))

sorted_similarity_matrix_score = sorted(similarity_matrix_score, key = lambda x: x[1], reverse = True)

    

In [ ]:
anime_master_df['episodes'] = anime_master_df['episodes'].fillna('')

In [ ]:
anime_cols = ['anime_id', 'anime_url', 'image_url', 'name', 'score', 'themes', 'demographics', 'synopsis', 'type', 'episodes', 'producers', 'source', 'combined_features']
animedf = anime_master_df[anime_cols]

In [ ]:
def to_lower(text):
    text = text.lower()
    return text
    
animedf['name'] = animedf['name'].apply(lambda x: to_lower(x))

In [ ]:
animedf.info()

In [28]:
# import pickle

# with open('artifacts/similarity_matrix.pkl', 'wb') as f:
#     pickle.dump(similarity, f)

In [31]:
import numpy as np
np.save("artifacts/similarity_matrix.npy", similarity.astype("float32"))

In [ ]:
animedf.to_csv('artifacts/anime_data.csv', index = False)

In [ ]:
import requests
import pandas as pd
url = 'https://api.jikan.moe/v4/seasons/now?limit=25'

response = requests.get(url)
data = response.json()
data['data']

new_anime_list = []
for item in data['data']:
    new_anime_list.append({
        'anime_id': item['mal_id'],
        'anime_url': item['url'],
        'image_url': item['images']['jpg']['image_url'],
        'name': item['title'],
        'score': item.get('score', None),
        'themes': [t["name"] for t in item.get("themes", [])],
        'demographics': [d["name"] for d in item.get("demographics", [])],
        'synopsis': item.get('synopsis', ''),
        'type': item.get('type', ''),
        'episodes': item.get("episodes", None),
        'producers': [p["name"] for p in item.get("producers", [])],
        'source': item.get("source", ""),
        'combined_features': None
    })

new_df = pd.DataFrame(new_anime_list)

In [ ]:
new_df = new_df.sort_values(by = 'score', ascending = False).head()
new_df

In [ ]:
# from datetime import datetime

# date_str = datetime.now().strftime("%d-%m-%Y")

new_df.to_csv('artifacts/trending_df.csv', index = False)

In [ ]:
animedf.sort_values(by = 'score', ascending = False).head()